In [1]:
import polars as pl
import numpy as np
import requests
import pandas as pd
from xgboost import XGBClassifier
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score
from numpy import argmax
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [2]:
url = 'https://storage.googleapis.com/open-courses/dmeyf2023-8a1e/competencia_02_crudo.csv.gz'
r = requests.get(url, allow_redirects=True)

In [3]:
with open('dataset.csv.gz', 'wb') as file:
    file.write(r.content)

In [4]:
!gunzip dataset.csv.gz

In [11]:
%%time

df = pl.read_csv('dataset.csv', infer_schema_length=10000)

CPU times: user 39.1 s, sys: 37.3 s, total: 1min 16s
Wall time: 1.55 s


In [12]:
min_val_dict = (
    df.select(
        pl.col(pl.Int64).min()
    )
).to_dict()
max_val_dict = (
    df.select(
        pl.col(pl.Int64).max()
    )
).to_dict()

bool_cols = []

uint8_cols = []
uint16_cols = []
uint32_cols = []
uint64_cols = []

int8_cols = []
int16_cols = []
int32_cols = []
int64_cols = []

for k in min_val_dict.keys():
    if min_val_dict[k][0]==0 and max_val_dict[k][0]==1:
        bool_cols.append(k)
    elif min_val_dict[k][0]>=0 and max_val_dict[k][0]<=255:
        uint8_cols.append(k)
    elif min_val_dict[k][0]>=0 and max_val_dict[k][0]<=65535:
        uint16_cols.append(k)
    elif min_val_dict[k][0]>=0 and max_val_dict[k][0]<=4294967295:
        uint32_cols.append(k)
    elif min_val_dict[k][0]>=0:
        uint64_cols.append(k)
    elif min_val_dict[k][0]>=-128 and max_val_dict[k][0]<=127:
        int8_cols.append(k)
    elif min_val_dict[k][0]>=-32768 and max_val_dict[k][0]<=32767:
        int16_cols.append(k)
    elif min_val_dict[k][0]>=-2147483648 and max_val_dict[k][0]<=2147483647:
        int32_cols.append(k)
    else:
        print(f'Int64? {k}, min={min_val_dict[k][0]}, max={max_val_dict[k][0]}')
        int64_cols.append(k)

In [13]:
all_int_cols = [bool_cols,
                uint8_cols,
                uint16_cols,
                uint32_cols,
                uint64_cols,
                int8_cols,
                int16_cols,
                int32_cols,
                int64_cols
               ]

[len(x) for x in all_int_cols]

[8, 44, 20, 2, 0, 0, 3, 2, 0]

In [14]:
df = df.with_columns(
    [
        pl.col(pl.Float64).cast(pl.Float32),
        pl.col(bool_cols).cast(pl.Boolean),
        pl.col(uint8_cols).cast(pl.UInt8),
        pl.col(uint16_cols).cast(pl.UInt16),
        pl.col(uint32_cols).cast(pl.UInt32),
        pl.col(uint64_cols).cast(pl.UInt64),
        pl.col(int8_cols).cast(pl.Int8),
        pl.col(int16_cols).cast(pl.Int16),
        pl.col(int32_cols).cast(pl.Int32),
        pl.col(int64_cols).cast(pl.Int64),
    ]
)
print(df.estimated_size())
df.write_parquet('crude_dataset.parquet')

1929811096


In [15]:
# df = pl.read_parquet('crude_dataset.parquet')

month_ranking = list(df.select(pl.col('foto_mes').unique())['foto_mes'])
month_ranking = {el:i for i, el in enumerate(month_ranking)}

In [16]:
x=(
    df
    .groupby("numero_de_cliente", maintain_order=True)
    .max()
    .select(
        [
            pl.col('numero_de_cliente'), 
            pl.col('foto_mes').alias('last_month')
        ]
    )
)
df = df.join(x, on="numero_de_cliente")

/tmp/ipykernel_2377117/1147081848.py:2: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  df


In [17]:
df = (
    df.select(
        [
            pl.all(),
            pl.col('foto_mes').apply(function=lambda x: month_ranking[x],return_dtype=pl.UInt8).alias('month_rk'),
            pl.col('last_month').apply(function=lambda x: month_ranking[x],return_dtype=pl.UInt8).alias('lm_rk')
        ]
    ).select(
        [pl.all(),
        (pl.col('lm_rk')-pl.col('month_rk')).alias('month_diff')]
    ).with_columns(
        pl.when(pl.col("foto_mes")==202107)
        .then('C')
        .when(pl.col("month_diff")==0)
        .then('B2')
        .when((pl.col("month_diff")==1) & (pl.col("foto_mes")!=202106))
        .then('B2')
        .otherwise('C')
        .alias("target")
    )
)

df = df.drop(['last_month','month_rk','lm_rk','month_diff'])

/tmp/ipykernel_2377117/1515679372.py:5: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  pl.col('foto_mes').apply(function=lambda x: month_ranking[x],return_dtype=pl.UInt8).alias('month_rk'),
/tmp/ipykernel_2377117/1515679372.py:5: PolarsInefficientMapWarning: 
Expr.map_elements is significantly slower than the native expressions API.
Only use if you absolutely CANNOT implement your logic otherwise.
In this case, you can replace your `map_elements` with the following:
  - pl.col("foto_mes").map_elements(lambda x: ...)
  + pl.col("foto_mes").map_dict(month_ranking)

  pl.col('foto_mes').apply(function=lambda x: month_ranking[x],return_dtype=pl.UInt8).alias('month_rk'),
/tmp/ipykernel_2377117/1515679372.py:6: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  pl.col('last_month').apply(function=lambda x: month_ranking[x],return_dtype=pl.UInt8).alias('lm_rk')
/tmp/ipykernel_2377117/1515679372.py:6: PolarsInefficientMapWar

In [19]:
df = df.with_columns(
    pl.when(pl.col(pl.Float32) != 0)
        .then(pl.col(pl.Float32))
        .keep_name()
)

In [20]:
float_cols = (
    df.select(
        pl.col(pl.Float32)
    )
).to_dict().keys()
float_cols = list(float_cols)

In [21]:
%%time

df = df.with_columns([
    pl.col(pl.Float32).shift(1).over("numero_de_cliente").suffix("_lag1"),
    pl.col(pl.Float32).shift(2).over("numero_de_cliente").suffix("_lag2"),
    pl.col(pl.Float32).shift(3).over("numero_de_cliente").suffix("_lag3"),
])

CPU times: user 29min 26s, sys: 26.1 s, total: 29min 52s
Wall time: 1min 9s


In [22]:
df.write_parquet('data_prepared.parquet')

In [34]:
%%time

# Dataset for hyperparameter tuning
X = df.select(pl.all().exclude('target'))
y = df.select([pl.col('target'), pl.col('foto_mes')])

X_val = X.filter((pl.col('foto_mes')==202002) | (pl.col('foto_mes')==202003) | (pl.col('foto_mes')==202004))
X_train = X.filter(pl.col('foto_mes')<=202001)

X_val = X_val.select(pl.all().exclude(['foto_mes', 'numero_de_cliente'])).to_numpy()
X_train = X_train.select(pl.all().exclude(['foto_mes', 'numero_de_cliente'])).to_numpy()

y_val = y.filter((pl.col('foto_mes')==202002) | (pl.col('foto_mes')==202003) | (pl.col('foto_mes')==202004))
y_train = y.filter(pl.col('foto_mes')<=202001)

y_val =  y_val.select(pl.all().exclude('foto_mes')).to_numpy()
y_train = y_train.select(pl.all().exclude('foto_mes')).to_numpy()

y_train = y_train.reshape(y_train.shape[0])
y_val = y_val.reshape(y_val.shape[0])
y_train = np.where(y_train=='B2', 1, 0)
y_val = np.where(y_val=='B2', 1, 0)

CPU times: user 19 s, sys: 1min 26s, total: 1min 45s
Wall time: 25 s


In [36]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((1717837, 371), (445324, 371), (1717837, 1), (445324, 1))

In [79]:
# Hyperparameter tuning

space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
    'gamma': hp.uniform ('gamma', 1,9),
    'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
    'reg_lambda' : hp.uniform('reg_lambda', 0,1),
    'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
    'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
    'n_estimators': hp.uniform('n_estimators', 500, 1200),
    'scale_pos_weight': hp.quniform("scale_pos_weight", 1, 25, 1),
}

def objective(space):
    clf=XGBClassifier(eval_metric="aucpr",
                    n_estimators = int(space['n_estimators']), 
                    max_depth = int(space['max_depth']), 
                    gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),
                    min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']),
                    scale_pos_weight = int(space['scale_pos_weight']), 
                    tree_method="hist",
     )
    
    evaluation = [( X_train, y_train), ( X_val, y_val)]
    
    clf.fit(X_train, y_train, eval_set=evaluation, early_stopping_rounds=10,verbose=False)
    
    probs = clf.predict_proba(X_val)
    preds = probs[:,1]
    
    fpr, tpr, threshold = metrics.roc_curve(y_val, preds, pos_label=1)
    roc_auc = metrics.auc(fpr, tpr)
    precision, recall, thresholds = metrics.precision_recall_curve(y_val, preds, pos_label=1)
    beta = 1.5
    fbeta = ((1+beta**2) * precision * recall) / ((beta**2) * precision + recall+0.000001)
    ix = argmax(fbeta)
    print(f'Best Threshold={thresholds[ix]}, F-Score={fbeta[ix]}, recall={recall[ix]}, precision={precision[ix]}')
    return {'loss': -fbeta[ix], 'status': STATUS_OK }

In [80]:
%%time

trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 150,
                        trials = trials)
print("The best hyperparameters are : ","\n")
print(best_hyperparams)

  0%|                                                                           | 0/150 [00:00<?, ?trial/s, best loss=?]

/home/ynesterov/miniconda3/lib/python3.9/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



Best Threshold=0.64475417137146, F-Score=0.24940448558955108, recall=0.368609865470852, precision=0.14436248682824027   
  1%|▎                                            | 1/150 [00:58<2:25:14, 58.49s/trial, best loss: -0.24940448558955108]

/home/ynesterov/miniconda3/lib/python3.9/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



Best Threshold=0.5598704814910889, F-Score=0.2533472420556383, recall=0.35112107623318384, precision=0.15575890192958028
  1%|▌                                             | 2/150 [01:55<2:21:46, 57.48s/trial, best loss: -0.2533472420556383]

...

Best Threshold=0.02383246086537838, F-Score=0.11829535366187913, recall=0.22869955156950672, precision=0.05670446964643095
 99%|██████████████████████████████████████████▍| 148/150 [2:09:46<01:40, 50.13s/trial, best loss: -0.26460675825104735]

/home/ynesterov/miniconda3/lib/python3.9/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



Best Threshold=0.3697376549243927, F-Score=0.25962412218046427, recall=0.3596412556053812, precision=0.15969733174034248
 99%|██████████████████████████████████████████▋| 149/150 [2:10:39<00:51, 51.23s/trial, best loss: -0.26460675825104735]

/home/ynesterov/miniconda3/lib/python3.9/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



Best Threshold=0.22974476218223572, F-Score=0.2631656465365139, recall=0.35336322869955156, precision=0.16716164616037335
100%|███████████████████████████████████████████| 150/150 [2:11:34<00:00, 52.63s/trial, best loss: -0.26460675825104735]
The best hyperparameters are :  

{'colsample_bytree': 0.7641758545981301, 'gamma': 6.424128698676173, 'max_depth': 5.0, 'min_child_weight': 0.0, 'n_estimators': 1023.0078555247765, 'reg_alpha': 86.0, 'reg_lambda': 0.48714545997765757, 'scale_pos_weight': 2.0}
CPU times: user 1d 17h 17min, sys: 10min 3s, total: 1d 17h 27min 4s
Wall time: 2h 11min 34s


In [81]:
%%time

# Dataset for the final model
X = df.select(pl.all().exclude('target'))
y = df.select([pl.col('target'), pl.col('foto_mes')])

X_test = X.filter(pl.col('foto_mes')==202107)
X_train = X.filter(pl.col('foto_mes')<=202106)

X_test = X_test.select(pl.all().exclude(['foto_mes', 'numero_de_cliente'])).to_numpy()
X_train = X_train.select(pl.all().exclude(['foto_mes', 'numero_de_cliente'])).to_numpy()

y_train = y.filter(pl.col('foto_mes')<=202106)
y_train = y_train.select(pl.all().exclude('foto_mes')).to_numpy()

y_train = y_train.reshape(y_train.shape[0])
y_train = np.where(y_train=='B2', 1, 0)

CPU times: user 29.7 s, sys: 1min 38s, total: 2min 8s
Wall time: 50.2 s


In [82]:
%%time

params1 = {'colsample_bytree': 0.7641758545981301, 
           'gamma': 6.424128698676173, 
           'max_depth': 5, 
           'min_child_weight': 0.0, 
           'n_estimators': 1024, 
           'reg_alpha': 86.0, 
           'reg_lambda': 0.48714545997765757, 
           'scale_pos_weight': 2,
           'tree_method': "hist"}

params2 = {'colsample_bytree': 0.7641758545981301, 
           'gamma': 6.424128698676173, 
           'max_depth': 5, 
           'min_child_weight': 0.0, 
           'n_estimators': 1024, 
           'reg_alpha': 86.0, 
           'reg_lambda': 0.48714545997765757, 
           'tree_method': "hist",
           'scale_pos_weight': 10}

xgb1 = XGBClassifier(**params1)
xgb1.fit(X_train, y_train)

xgb2 = XGBClassifier(**params2)
xgb2.fit(X_train, y_train)

CPU times: user 8h 2min 23s, sys: 37 s, total: 8h 3min
Wall time: 7min 38s


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7641758545981301, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=6.424128698676173,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=0.0, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1024, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [131]:
sub_base = df.filter(pl.col('foto_mes')==202107).select(pl.col('numero_de_cliente')).to_pandas()
sub_xgb1 = sub_base.copy()
sub_xgb1['Predicted'] = np.where(preds_test1>=0.06, 1, 0)
sub_xgb1.to_csv('xgb_spw1_006t.csv', index=False)